In [1]:
lable = {'Abyssinian': 0,
 'american_bulldog': 1,
 'american_pit_bull_terrier': 2,
 'basset_hound': 3,
 'beagle': 4,
 'Bengal': 5,
 'Birman': 6,
 'Bombay': 7,
 'boxer': 8,
 'British_Shorthair': 9,
 'chihuahua': 10,
 'Egyptian_Mau': 11,
 'english_cocker_spaniel': 12,
 'english_setter': 13,
 'german_shorthaired': 14,
 'great_pyrenees': 15,
 'havanese': 16,
 'japanese_chin': 17,
 'keeshond': 18,
 'leonberger': 19,
 'Maine_Coon': 20,
 'miniature_pinscher': 21,
 'newfoundland': 22,
 'Persian': 23,
 'pomeranian': 24,
 'pug': 25,
 'Ragdoll': 26,
 'Russian_Blue': 27,
 'saint_bernard': 28,
 'samoyed': 29,
 'scottish_terrier': 30,
 'shiba_inu': 31,
 'Siamese': 32,
 'Sphynx': 33,
 'staffordshire_bull_terrier': 34,
 'wheaten_terrier': 35,
 'yorkshire_terrier': 36}

In [26]:
def accuracy(output, target, topk=(1,)):
    """
        计算topk的准确率
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        class_to = pred[0].cpu().numpy()

        res = []
        for k in topk:
            correct_k = correct[:k].contiguous().view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res, class_to

In [3]:
# import torchvision,torch,torch.nn as nn
# model = torchvision.models.vit_b_16(progress=True)
# # model = torchvision.models.vit_b_16(progress=True)
# # model = vit_model.vit_base_patch16_224(num_classes=37)
# model.heads = nn.Sequential(nn.Linear(768,out_features=37,bias=True))
# stact = torch.load(r'D:\code\EML-NET-Saliency-master\model_best_checkpoint_resnet50.pth.tar',map_location='cpu')
# model.load_state_dict(stact['state_dict'])
from PIL import Image
import torchvision.transforms as tr
import torch

In [63]:
path = r'D:\data\animals_catalogy\data\1-1.jpg'
path_ = r'D:\data\animals_catalogy\data\1-1-1.jpg'
trs = tr.Compose([
    tr.ToTensor(),
    # tr.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
input = trs(Image.open(path).convert('RGB').resize((224,224)))
input_two = trs(Image.open(path_).convert('RGB').resize((224,224)))
# model(input.unsqueeze_)
input_ = torch.unsqueeze(input,0)
input_two_ = torch.unsqueeze(input_two,0)
# output= model(input_)
# accuracy()
# input_.shape
# _, pred = output.topk(1, 1, True, True)
# [k for k, v in lable.items() if v == pred][0]

In [61]:
from timm.models.vision_transformer import PatchEmbed, Block
patch_embed = PatchEmbed()
a = patch_embed(input_)
b = patch_embed(input_two_)  ### 中间这个表示每一块的 16* 16
# c = a==b 
input_two.shape

torch.Size([3, 224, 224])

In [35]:
import torch.nn as nn 
import torch.nn.functional as F
proj = nn.Conv2d(3, 768, kernel_size=16, stride=16, bias=False) ### 1,3 ,224 ,224 => 1,768,14,14

In [83]:
a_ = torch.ones(768,3,16,16)
a_ = torch.nn.Parameter(a_)
a_ = a_.to(dtype=torch.float32)
proj.weight = a_
y = proj(input_)
torch.mul(x,y)

tensor([[[[768.0000, 768.0000, 756.5648,  ..., 768.0000, 768.0000, 768.0000],
          [768.0000, 768.0000,   0.0000,  ...,   0.0000,   0.0000, 768.0000],
          [768.0000, 768.0000,   0.0000,  ...,   0.0000,   0.0000, 768.0000],
          ...,
          [768.0000, 768.0000,   0.0000,  ...,   0.0000,   0.0000, 768.0000],
          [768.0000, 768.0000, 768.0000,  ..., 768.0000, 768.0000, 768.0000],
          [768.0000, 768.0000, 768.0000,  ..., 768.0000, 768.0000, 768.0000]],

         [[768.0000, 768.0000, 756.5648,  ..., 768.0000, 768.0000, 768.0000],
          [768.0000, 768.0000,   0.0000,  ...,   0.0000,   0.0000, 768.0000],
          [768.0000, 768.0000,   0.0000,  ...,   0.0000,   0.0000, 768.0000],
          ...,
          [768.0000, 768.0000,   0.0000,  ...,   0.0000,   0.0000, 768.0000],
          [768.0000, 768.0000, 768.0000,  ..., 768.0000, 768.0000, 768.0000],
          [768.0000, 768.0000, 768.0000,  ..., 768.0000, 768.0000, 768.0000]],

         [[768.0000, 768.0000,

In [87]:
a_ = torch.ones(768,3,16,16)
a_ = torch.nn.Parameter(a_)
a_ = a_.to(dtype=torch.float32)
proj.weight = a_
x = proj(input_two_)
# F.conv2d(input=input_two,weight=weight,stride=2,padding='valid')
# weight
# a_
# x= x.to(dtype=torch.int)
x = x > 0
x = x.to(dtype=torch.float)
# x = torch.unsqueeze(x,0)
x.shape

torch.Size([1, 768, 14, 14])

In [32]:
import collections
from itertools import repeat
def _ntuple(n):
    def parse(x):
        if isinstance(x, collections.abc.Iterable) and not isinstance(x, str):
            return tuple(x)
        return tuple(repeat(x, n))
    return parse
to_2tuple = _ntuple(2)
to_2tuple(21)
to_2tuple(224)
grid_size = tuple([s // p for s, p in zip(to_2tuple(224), to_2tuple(16))])
grid_size### 每个窗格大小是14 * 14 步长是16  卷积核是16 

(14, 14)